In [ ]:
import pandas as pd
import numpy as np
import random
import glob
import tempfile
import cv2
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.preprocessing import LabelEncoder
import torch
from torch import nn, optim, Tensor
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchinfo import summary
import timm
import torchvision
from torchvision import transforms, datasets, models
import torch.utils.data as Data
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split
from matplotlib import pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train = pd.read_csv(r"/kaggle/input/UBC-OCEAN/train.csv")
test = pd.read_csv(r"/kaggle/input/UBC-OCEAN/test.csv")
train_tma = train[train["is_tma"] == True]
train_no_tma = train[train["is_tma"] == False]

In [ ]:
train_tma['img_id_ext'] = [str(i) + " .png" for i in train_tma['image_id']]
train_no_tma['img_id_ext'] = [str(i) + "_thumbnail.png" for i in train_no_tma['image_id']]
test['img_id_ext'] = [str(i) + "_thumbnail.png" for i in test['image_id']]

In [ ]:
train_df = pd.concat([train_tma, train_no_tma])
train_df.sort_index(ascending = True, inplace = True)
train_df.head()

In [ ]:
le = LabelEncoder()
image_label = train_df['label']
le.fit(image_label)
train_df['label'] = le.transform(image_label)
train_df.head()

In [ ]:
import os

class TrainDataset(Dataset):
    def __init__(self, df, root_path, train_thumbnails, train_images, transform = None):
        self.df = df
        self.root_path = root_path
        self.train_thumbnails = train_thumbnails
        self.train_images = train_images
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_name = self.df.iloc[idx, -1]
        
        if "thumbnail" in img_name:
            img_path = os.path.join(root_path + train_thumbnails, img_name)
        else:
            img_path = os.path.join(root_path + traing_images, img_name)
        
        img = Image.open(img_path).convert('RGB')
        
        if self.transform:
            img = self.transform(img)
        
        label = self.df.iloc[idx, 1]
        label = torch.tensor(label)
        return img, label

norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]

train_transform = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.CenterCrop(224), 
    transforms.RandomHorizontalFlip(p = 0.5), 
    transforms.RandomVerticalFlip(p = 0.5),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

valid_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(norm_mean, norm_std),
])

root_path = "/kaggle/input/UBC-OCEAN/"
train_thumbnails = "train_thumbnails/"
train_images = "train_images/"
test_thumbnails = "test_thumbnails/"
test_images = "test_images/"

dataset = TrainDataset(df = train_df, root_path = root_path, train_thumbnails = train_thumbnails, train_images = train_images, transform = train_transform)

train_ratio = 0.8
valid_ratio = 0.2

train_size = int(train_ratio * len(train_df))
valid_size = len(train_df) - train_size

dataset_train, dataset_valid = random_split(dataset, [train_size, valid_size])

train_dataloader = DataLoader(dataset_train, batch_size = 8, shuffle = False, num_workers = 5)
valid_dataloader = DataLoader(dataset_train, batch_size = 8, shuffle = False, num_workers = 5)

dataiter = iter(train_dataloader)
images = next(dataiter)
images[0].shape